In [2]:
!pip install transformers
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 27.0 MB/s eta 0:00:0000:0100:01


  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.22.3 which is incompatible.


In [ ]:
from transformers import InputExample, InputFeatures
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [ ]:
import tensorflow as tf
import pandas as pd
df = pd.read_json('Data/review_1000.json', lines=True)

#get only the text
#sentences = df.text.values

#assigns positive of negative (words) to sentiment
df["Sentiment"] = df.stars.apply(lambda score: "positive" if score >= 3 else "negative")
# change positive / negative to 1/ 0
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['ratings'] = df.stars
#just show the review and the new 1/0 
df = df[["text", "Sentiment", "ratings"]]

print(df)

In [4]:
reviews = df.text.values.tolist()
labels = df.Sentiment.tolist()
stars = df.ratings.tolist()



In [5]:
print(reviews[:2])
print(labels[:2])

["If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nFor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nThere is no way I can write this review without giving Russell, th

In [6]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [7]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [8]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 15180, 3993, 2140, 999, 999, 999, 999, 2065, 1045, 2071, 2507, 5717, 2030, 4997, 3340, 1045, 2052, 999, 1045, 1005, 2310, 2196, 2626, 1037, 3319, 2077, 1010, 2021, 1045, 2572, 2183, 2041, 1997, 2026, 2126, 2000, 2360, 2008, 1045, 2018, 1996, 7619, 5409, 7759, 3325, 2012, 1996, 8739, 2160, 999, 999, 1998, 9686, 2361, 2945, 1010, 2994, 3154, 999, 2061, 2034, 2125, 1010, 1045, 2787, 2000, 2202, 2026, 2321, 3204, 2214, 2365, 2041, 2006, 1037, 10514, 6182, 3058, 1012, 2057, 7180, 2012, 1996, 8739, 2160, 2012, 1018, 9737, 1012, 2045, 2003, 2069, 2028, 2060, 2795, 1998, 2027, 2020, 5494, 2041, 1012, 2057, 4133, 2091, 1010, 1998, 1045, 3202, 3198, 2005, 1037, 3722, 3256, 2300, 2007, 14380, 1012, 15610, 3310, 2067, 2322, 2781, 2101, 1006, 2053, 2300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))


2022-04-15 07:18:38.772714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-15 07:18:38.772762: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-15 07:18:38.772783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cet-herndon-jupyter): /proc/driver/nvidia/version does not exist
2022-04-15 07:18:38.773093: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)


2022-04-15 07:18:39.811764: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

In [12]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [13]:
model.save_pretrained("./sentiment")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")


Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
totalPos = 0
totalNeg = 0

for i in stars:
    if (i >= 3):
        totalPos = totalPos + 1
    else: 
        totalNeg = totalNeg + 1
# count the number of positives and negatives based on numbers
print(len(reviews))
print(totalPos)
print(totalNeg)

1000
814
186


In [15]:
labelString = ['Negative','Positive']

# test each review
# middle_index=len(reviews)//2

# first_half=reviews[:middle_index]
# sec_half=reviews[middle_index:]
# label_first_half = labels[:middle_index]
# label_sec_half = labels[middle_index:]
index = 0
currPos = 0
currNeg = 0
for item1 in reviews:
    # compare to item1 in second half
    predict_input = tokenizer.encode(item1,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
   
    tf_output = loaded_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    label = tf.argmax(tf_prediction, axis=1)
    label = label.numpy()
    
    if (label[0] == 0):
        currNeg = currNeg + 1
    else:
        currPos = currPos + 1
    index = index + 1
    

print(index)
print(currPos)
print(currNeg)
 

1000
441
559
